In [1]:
import time
import warnings
from multiprocessing import Pool
from pathlib import Path
from typing import Optional
from pathlib import Path
from typing import Mapping, Collection, Literal

from dustgoggles.mosaic import parquet_metadata_records, meta_record_df, meta_column_df
import matplotlib.pyplot as plt
import matplotlib as mpl
from marslab.imgops.imgutils import std_clip
from marslab.imgops.pltutils import strip_axes
from fitsio import FITS
from more_itertools import windowed
import numpy as np
import pandas as pd
import pyarrow
from pyarrow import parquet
from pympler.asizeof import asizeof

from gPhoton.aspect import aspect_tables
from gPhoton.aspect import load_aspect_solution
from gPhoton.calibrate import find_fuv_offset
from gPhoton.coords.gnomonic import gnomfwd_simple

from gPhoton.io.mast import retrieve_raw6
from gPhoton.io.raw6 import load_raw6, get_eclipse_from_header
from gPhoton.photonpipe._steps import (
    create_ssd_from_decoded_data,
    perform_yac_correction,
    load_cal_data,
    process_chunk_in_shared_memory,
    chunk_data,
    process_chunk_in_unshared_memory,
)
from gPhoton.parquet_utils import parquet_to_ndarrays
from gPhoton.pretty import print_inline
from gPhoton.search.core import filter_galex_eclipses
from gPhoton.sharing import (
    unlink_nested_block_dict,
    slice_into_shared_chunks,
    send_mapping_to_shared_memory,
    get_column_from_shared_memory,
)
from gPhoton.reference import eclipse_to_paths
from gPhoton.types import GalexBand, Pathlike

%matplotlib qt
mpl.rcParams['font.size'] = 24
mpl.rcParams['image.cmap'] = 'Greys_r'

In [1]:
from pyarrow import parquet
import pandas as pd 

In [5]:
ais_eclipses = filter_galex_eclipses(filters=[("obstype", "=", "AIS")])
# examples = (15943, 45845)
eclipse, band = 15943, "NUV"
paths = eclipse_to_paths(
    eclipse, data_directory="test_data", compression='rice'
)[band]

In [9]:
cnt = FITS(paths['images'][0])[1].read()

In [11]:
plt.imshow(std_clip(cnt))

In [ ]:
image_files = eclipse_to_paths(
    23456, compression='rice', data_directory='test_data'
)["NUV"]['images']
# fig, grid = plt.subplots(3, 4)
# for ix, image_file in enumerate(image_files):
#     cnt = FITS(image_file)[1].read()
#     ax = grid[ix % 3][ix % 4]
#     ax.imshow(std_clip(cnt))
#     strip_axes(ax)
# fig.show()


In [ ]:
cnt = FITS(image_files[0])[1].read()
plt.imshow(std_clip(cnt))

In [ ]:
cnt_old = FITS(
    "test_data/e23456/e23456-nd-full-rice.fits"
)[1].read()

In [ ]:
plt.imshow(std_clip(np.abs(cnt_old - cnt), sigma=50))
plt.colorbar()

In [ ]:
len(cnt[np.isfinite(cnt)])

In [ ]:
len(cnt_old[np.isfinite(cnt_old)])

In [ ]:
meta_column_df(eclipse_to_paths(23456, data_directory="test_data")["NUV"]["photonfiles"][0])

In [ ]:
meta_column_df(eclipse_to_paths(23456, data_directory="test_data")["NUV"]["photonfiles"][0])

In [ ]:
143290187/286584963

In [ ]:
plt.imshow(std_clip(cnt))

In [ ]:
fig.tight_layout()

In [ ]:
a = {
    (0, 1): 1,
    (0, 2): 2,
    (1, 0): 1,
    (1, 1): 2
}    

In [ ]:
b = {
    address: a.pop(address)
    for address in a.keys()
    if address[0] == 0
}

In [3]:
parquet.read_table(
            '/home/bekah/gPhoton2/test_data/e00580/e00580-nd-b00.parquet', columns=['t', 'col', 'row', 'detrad']
        )

pyarrow.Table
t: double
col: float
row: float
detrad: float
----
t: [[738997337.995,738997337.995,738997337.995,738997337.995,738997337.995,...,738997341.8,738997341.8,738997341.8,738997341.8,738997341.8],[738997341.8,738997341.8,738997341.8,738997341.8,738997341.8,...,738997345.585,738997345.585,738997345.585,738997345.585,738997345.585],...,[738997891.595,738997891.595,738997891.595,738997891.595,738997891.595,...,738997895.56,738997895.56,738997895.56,738997895.56,738997895.56],[738997895.56,738997895.56,738997895.56,738997895.56,738997895.56,...,738997898.99,738997898.99,738997898.99,738997898.99,738997898.99]]
col: [[301.0346,406.84717,279.5562,215.63876,399.0325,...,318.20795,155.94112,764.90106,403.56775,713.57556],[559.8968,479.69144,111.23763,106.29762,754.9899,...,337.36295,407.38724,300.7121,516.6921,418.53616],...,[354.10757,354.9103,155.0092,203.93974,278.1068,...,175.79214,440.03006,247.48195,497.46524,434.89377],[44.946453,502.5863,181.69638,455.0482,306.1436,...,207.197

In [1]:
from pyarrow import parquet 


In [12]:
#bore
bore = parquet.read_table("/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet",filters=[('eclipse','==',580)]).to_pandas()

In [17]:
bore['time'].iloc[0]

738997337.995

In [6]:
tab = parquet.read_table("/home/bekah/gPhoton2/gPhoton/aspect/aspect2.parquet",filters=[('eclipse','==',580)]).to_pandas()

In [9]:
tab["pktime"].iloc[0]

738997305.995

In [11]:
tab[(tab['hvnom_fuv'] == 1) | (tab['hvnom_nuv'] == 1)]['pktime'].iloc[0]

738997337.995

In [23]:
tab.iloc[0]['pktime']-tab1.iloc[0]['time']

0.0

In [19]:
738997305.995-738997337.995-559


-591.0

In [80]:
738997305.995-738997337.995


-32.0

In [68]:
tab.iloc[0]['pktime']

738997337.995

In [ ]:
738997337.995

In [76]:
tab.iloc[564]['pktime']-tab.iloc[0]['pktime']

564.0

In [22]:
tab.iloc[558]['pktime']-tab1.iloc[564]['time']

-38.0

In [23]:
tab.iloc[0]['pktime']-tab1.iloc[0]['time']

-32.0

In [9]:
tab.iloc[557]['pktime']

738997899.995

In [11]:
tab=tab.rename({'pktime':'time'})
tab = tab.join(tab1,'time')
tab

KeyError: 'time'

In [22]:
tab1 = parquet.read_table("/home/bekah/gPhoton2/gPhoton/aspect/aspect.parquet",filters=[('eclipse','==',580)]).to_pandas()

In [5]:
tab1.iloc[0]['time']-738997337.995

0.0

In [2]:
bore = parquet.read_table("/home/bekah/gPhoton2/gPhoton/aspect/boresight.parquet",filters=[('eclipse','==',580)]).to_pandas()

In [50]:
float(bore['time'])

738997337.995

In [3]:
bore

,eclipse,time,ra0,dec0,roll0
0,580,7.389973e+08,210.554346,-32.686215,0.000632


In [52]:
tab1.iloc[0]['time']

738997337.995

In [57]:
tab.iloc[0]['pktime']

738997342.995

In [59]:
tab.iloc[0]['pktime'] 
# no voltage filter 

738997305.995

In [61]:
tab.iloc[0]['pktime']-float(bore['time'])

-32.0

In [62]:
738997342.995-float(bore['time'])

5.0

In [12]:
tab1.iloc[0]['time']-tab.iloc[0]['pktime']

32.0

In [43]:
tab1.iloc[1316]['time']-tab1.iloc[0]['time']

1699.0

In [42]:
tab1.iloc[1316]['time']-tab.iloc[1702]['pktime']

-3.0

In [ ]:
raw6_tab = pyarrow.Table.from_arrays(
    tuple(data.values()), names=tuple(data.keys())
)

In [ ]:
splits = [
    0, 
    *np.nonzero(np.diff(aspect['time']) > 10)[0], 
    len(aspect['time'])
]
partitions = list(windowed(splits, 2))
asp_df = pd.DataFrame(aspect)
legs = [
    asp_df.iloc[p[0] + 1:p[1] + 1] for p in partitions
]

In [ ]:
len(aspect['eclipse'])

In [ ]:
aspect = parquet.read_table(
    '/home/michael/Desktop/gphoton_working/gPhoton/aspect/aspect.parquet',
    filters=[('eclipse', '=', eclipse)]
).to_pandas()
partitions = [(-1, partitions[0][0]), *partitions, (partitions[-1][1], -2)]


In [ ]:
parquet.write_table(
    raw6_tab, 
    "unpacked_raw6.parquet", 
    use_dictionary=["t", "xb", "xamc", "yb", "yamc", "q", "xa", "ya", "x", "y"]
)

In [ ]:
meta_column_df('unpacked_raw6.parquet')

In [ ]:
import fitsio

In [ ]:
hdul = fitsio.FITS("/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd-full-0-rice.fits")

In [ ]:
hdul_2 = fitsio.FITS("/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd-full-rice.fits")

In [ ]:
cntnew = hdul[1].read()
cntold = hdul_2[1].read()

In [ ]:
import numpy as np
np.nanmax(np.abs(cntnew - cntold))

In [ ]:
import matplotlib.pyplot as plt
from marslab.imgops.imgutils import std_clip
%matplotlib qt

In [ ]:
plt.figure()
plt.imshow(np.abs(cntnew - cntold))
plt.colorbar()

In [ ]:
meta_column_df('/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd-0.parquet').T

In [ ]:
meta_column_df('/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd.parquet').T

In [ ]:
from pyarrow import parquet
tab_new = parquet.read_table(
    '/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd-0.parquet'
).to_pandas()

In [ ]:
tab_old = parquet.read_table(
    '/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd-0 (copy).parquet'
).to_pandas()

In [ ]:
plt.plot(tab_new['eta'] - tab_old['eta'])

In [ ]:
(tab_new['eta'] - tab_old['eta']).argmax()

In [ ]:
cut_new = tab_new.loc[34945920:34945925]

In [ ]:
cut_old = tab_old.loc[34945920:34945925]

In [ ]:

cut_new

In [ ]:
cut_old.dtypes

In [ ]:
cut_new.loc[34945923]['t']

In [ ]:
cut_old.loc[34945923]['t']

In [ ]:
tab_new.loc[tab_new['flags'] == 12]

In [ ]:
tab_old.loc[tab_old['flags'] == 12]

In [ ]:
cols = cut_new.dtypes.loc[cut_new.dtypes != bool]

In [ ]:
diff = cut_old[cols.index] - cut_new[cols.index]

In [ ]:
import gPhoton.constants as c

In [ ]:
tab2.loc[(tab2['response'] - tab['response']).abs() > 0.1]

In [ ]:
tab.loc[(tab2['ra'] - tab['ra']).abs() > 0.0001]

In [ ]:
(tab2['detrad'] - tab['detrad']).abs().argmax()

In [ ]:
(tab_new["ya"] - tab_old["ya"]).abs().max()

In [ ]:
import pandas as pd

In [ ]:
t = pd.read_csv(
    '/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd-full-0-photom-12_8.csv'
)

In [ ]:
a = pd.read_csv(
    '/home/michael/Desktop/gphoton_working/test_data/e23456/e23456-nd-full-photom-12_8.csv'
)

In [ ]:
def coregister_sources(xy0, xy1):
    matches = []
    for source_ix in range(xy0.shape[0]):
        vec = xy0[source_ix]
        vec = np.expand_dims(vec, 1).T
        offsets = np.linalg.norm(xy1 - vec, axis=1)
        matches.append(offsets.argmin())
    sort_xy1 = xy1[matches]
    return np.linalg.norm(sort_xy1 - xy0, axis=1)


def check_coregistration(ref, comp, threshold=1):
    c_cols = ['xcentroid', 'ycentroid']
    ref_xy = ref[c_cols].to_numpy()
    comp_xy = comp[c_cols].to_numpy()
    ref_offsets = coregister_sources(ref_xy, comp_xy)
    comp_offsets = coregister_sources(comp_xy, ref_xy)
    return (
        ref_offsets > threshold, comp_offsets > threshold
    )

In [ ]:
from gPhoton.pipeline import execute_pipeline

if __name__ == "__main__":
    execute_pipeline(
        23456,
        "NUV",
        depth=None,
        threads=4,
        local_root="test_data",
        recreate=True,
        aperture_sizes=[12.8],
        write={"movie": True, "image": True},
        coregister_lightcurves=True,
        # stop_after="moviemaker",
        compression="rice",
        lil=True
    )


In [ ]:
def coregister_sources_2(xy0, xy1):
    matches = []
    for source_ix in range(xy0.shape[0]):
        vec = xy0[source_ix]
        vec = np.expand_dims(vec, 1).T
        offsets = np.linalg.norm(xy1 - vec, axis=1)
        matches.append(offsets.argmin())
    sort_xy1 = xy1[matches]
    return matches, np.linalg.norm(sort_xy1 - xy0, axis=1)

In [ ]:
matches_t, offsets_t = coregister_sources_2(
    a[['xcentroid', 'ycentroid']].to_numpy(), t[['xcentroid', 'ycentroid']].to_numpy()
)
matches_a, offsets_a = coregister_sources_2(
    t[['xcentroid', 'ycentroid']].to_numpy(), a[['xcentroid', 'ycentroid']].to_numpy()
)

In [ ]:
mismatch_t = np.nonzero(offsets_t > 0.5)[0]
matchy_t = [m for ix, m in enumerate(matches_t) if ix not in mismatch_t]
mismatch_a = np.nonzero(offsets_a > 0.5)[0]
matchy_a = [m for ix, m in enumerate(matches_a) if ix not in mismatch_a]

In [ ]:
a = a.drop(mismatch_t).loc[matchy_a].reset_index(drop=True)
t = t.drop(mismatch_a).reset_index(drop=True)

In [ ]:
plt.plot(a['aperture_sum'] / t['aperture_sum'])

In [ ]:
plt.plot(a['ycentroid'] - t['ycentroid'])

In [ ]:
t.loc[np.isinf(t['aperture_sum'])]